In [35]:
import pandas as pd
import numpy as np
import requests
import json
from os import walk
from PIL import Image
from io import BytesIO
import os

In [2]:
df = pd.read_csv('../data/data.csv')

C:\Users\PC\AppData\Local\Temp\ipykernel_16680\785670083.py:1: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/data.csv')


In [3]:
df.index[0]

0

In [4]:
df.columns

Index(['image', 'Nameboard Image', 'Outlet Image', 'Remarks2', 'Date2',
       'DistributorCode', 'DistributorName', 'SalesRep', 'repcode',
       'OutletCode', 'OutletName', 'routename', 'RepName', 'RepId',
       'Longitude', 'Latitude', 'IsGPSOn', 'finnaly accepted or not'],
      dtype='object')

In [5]:
image_dir = '../data/images'

In [6]:
img_dir_lis = list()
for dirpath, dirname, filename in walk(image_dir):
    img_dir_lis.extend(filename)

len(img_dir_lis)

100

In [7]:
"""get row and index"""

df.index[0], df.loc[df.index[0], 'image']

(0,
 'https://surge.keells.lk/Myfiles/RetailerImages/1c62479b-12b9-4fac-a9bf-12f6f80f9269.jpg')

In [8]:
df['image'] = df['image'].apply(lambda x: 'https://surgepilot.lk/' + x.split('/')[-1])
df.head()


,image,Nameboard Image,Outlet Image,Remarks2,Date2,DistributorCode,DistributorName,SalesRep,repcode,OutletCode,OutletName,routename,RepName,RepId,Longitude,Latitude,IsGPSOn,finnaly accepted or not
0,https://surgepilot.lk/1c62479b-12b9-4fac-a9bf-...,Available: Image Fully Captured,outlet image partially acceptable,NaN,8/2/2024,0000052005,Geethika Distributors,Pettah Halal GT S Thileeban (G) - 141497,52005KFPREP06,KF041483,Benadic Stores,Jem.Street,Pettah Halal GT S Thileeban (G) - 141497,52005KFPCSR06,79.85769,6.94318,TRUE,Accept
1,https://surgepilot.lk/abf6fc21-6a21-4d48-91cc-...,Available: Image Fully Captured,outlet image partially acceptable,NaN,8/2/2024,0000011660,Ananda Distributors,U.G.S.Kosala Ranasinghe - Horana,R431,IC127389,Greenazy Hotel,"Wataraka,Meegoda,Dampe",U.G.S.Kosala Ranasinghe 11660,DSR000991,80.04332,6.82500,TRUE,Accept
2,https://surgepilot.lk/935c6299-29b9-42c9-80b8-...,Available: Image Fully Captured,outlet image partially acceptable,NaN,8/2/2024,0000066858,Rajatha Distributors (Pvt) Ltd,M.Lasantha Harshapriya - Baththaramulla (RD),REP0795,SD323031,Ranji & Deepika stores,Kirulapane 01,M.Lasantha Harshapriya 66858,DSR0001360,79.87400,6.87875,TRUE,Accept
3,https://surgepilot.lk/c289bcc1-7f83-4dee-a4ff-...,Available:Image Partially Captured,outlet image partially acceptable,NaN,8/2/2024,0000058673,Chamith Enterprises\t,U G Nissanka - Katana,58673SDDSR05,SD348814,SACHITH MINI MART,DSR 2 - 58673 - Route 14,U G Nissanka 58673,DSR001767,79.90961,7.20397,TRUE,Accept
4,https://surgepilot.lk/dd0b2b74-52b5-4a3f-ade2-...,Not Available,outlet image partially acceptable,NaN,8/2/2024,0000011736,Yasantha Distributors,LALINDA BANDUSEKARA - Polgasowita (YD),REP0279,SD289295,Rukmal stores,Pamunuwa & Temple rd,LALINDA BANDUSEKARA 11736,DSR000474,79.93596,6.85309,TRUE,Accept


In [9]:
df['image'].to_list()[0]

'https://surgepilot.lk/1c62479b-12b9-4fac-a9bf-12f6f80f9269.jpg'

In [37]:
def update_json(json_dir, ind, pred):
    """
    Updates a JSON file with the given index and prediction.

    Parameters:
    - json_dir (str): Path to the JSON file.
    - ind (int): Index value to be updated.
    - pred (str or dict or list): Prediction value to be updated.

    The function ensures:
    - JSON validity before writing.
    - Proper file handling with exceptions.
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(json_dir), exist_ok=True)

        # Data to be written
        data = {"index": ind, "prediction": pred}

        # Writing data to JSON file
        with open(json_dir, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        print(f"Successfully updated {json_dir}")

    except OSError as e:
        print(f"Error handling file {json_dir}: {e}")
    except TypeError as e:
        print(f"Invalid data type for JSON: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

In [ ]:
def load_json(json_dir):
    """
    Loads a JSON file and returns its contents.

    Parameters:
    - json_dir (str): Path to the JSON file.

    Returns:
    - dict: Parsed JSON data if successful.
    - None: If an error occurs.

    Exception Handling:
    - Handles file not found, JSON decoding errors, and permission issues.
    """
    try:
        with open(json_dir, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found - {json_dir}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_dir}")
    except PermissionError:
        print(f"Error: Permission denied for {json_dir}")
    except Exception as e:
        print(f"Unexpected error while loading {json_dir}: {e}")

    return None  # Return None if an error occurs

In [22]:
json_dir = '../data/pred.json'

In [26]:
json_data = load_json(json_dir)
print(json_data['index'])
print(json_data['prediction'])

[0, 1]
['a', 'b']


In [36]:
def download_images(save_dir, dir_lis, img_lis):
    """
    Downloads images from a list of URLs and saves them to the specified directory.

    Parameters:
    - save_dir (str): Directory where images will be saved.
    - dir_lis (list): List of existing image filenames in the directory.
    - img_lis (list): List of image URLs to be downloaded.

    Exceptions are handled for:
    - Connection errors
    - HTTP errors (e.g., 404, 403)
    - Invalid image format
    - Issues with saving the file
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i, img_url in enumerate(img_lis):
        try:
            img_name = img_url.split("/")[-1]

            if img_name in dir_lis:
                print(f"Skipping {img_name}, already exists.")
                continue

            response = requests.get(img_url, timeout=10)
            response.raise_for_status()  # Raises HTTPError for bad responses (4xx, 5xx)

            try:
                img = Image.open(BytesIO(response.content))
                img.save(os.path.join(save_dir, img_name))
                print(f"Successfully downloaded: {img_name}")
            except (OSError, IOError) as e:
                print(f"Error saving image {img_name}: {e}")

        except requests.exceptions.MissingSchema:
            print(f"Invalid URL: {img_url}")
        except requests.exceptions.ConnectionError:
            print(f"Failed to connect: {img_url}")
        except requests.exceptions.Timeout:
            print(f"Timeout error: {img_url}")
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error for {img_url}: {e}")
        except Exception as e:
            print(f"Unexpected error for {img_url}: {e}")

In [39]:
def dir_image_lis(path):
    """
    Retrieves a list of image filenames in a directory.

    Parameters:
    - path (str): Path to the directory.

    Returns:
    - list: List of image filenames in the directory.

    Exception Handling:
    - Handles cases where the directory does not exist.
    - Ensures only valid image files are returned.
    """
    try:
        if not os.path.exists(path):
            print(f"Error: Directory not found - {path}")
            return []

        img_lis = []
        valid_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}

        for root, _, files in os.walk(path):
            img_lis.extend(
                [
                    file
                    for file in files
                    if os.path.splitext(file)[1].lower() in valid_extensions
                ]
            )

        return img_lis

    except Exception as e:
        print(f"Unexpected error while reading directory {path}: {e}")
        return []

In [31]:
save_dir = '../data/images'

In [63]:
def get_prediction(api, image):
    """
    Sends an image to the API for prediction.

    Parameters:
    - api (str): API endpoint URL.
    - image (str): Path to the image file.

    Returns:
    - str or dict: Prediction result from the API.

    Exception Handling:
    - Handles missing files, API request failures, and unexpected errors.
    """
    try:
        with open(image, "rb") as img_file:  # Ensures file is properly closed after use
            response = requests.post(api, files={"file": img_file}, timeout=50)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)

        return response.json().get("prediction", "No prediction key in response")

    except FileNotFoundError:
        print(f"Error: Image file not found - {image}")
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except requests.exceptions.JSONDecodeError:
        print("Error: Unable to parse JSON response from API.")
    except Exception as e:
        print(f"Unexpected error: {e}")

    return None  # Return None if an error occurs

In [ ]:
def get_pred(next_ind, api_url, save_dir, json_dir):

    img_dir_lis = dir_image_lis(save_dir)
    json_data = load_json(json_data)

    ind_lis = json_data['index']
    pred_lis = json_data['prediction']
    last_ind = ind_lis[-1]
    
    if next_ind > last_ind:
        # Download images and predict for each image in the data frame
        for i in range(last_ind + 1, next_ind):

            image_link = df.loc[i, "image"]
            img_name = image_link.split("/")[-1]

            # Check if image is already downloaded
            download_images(save_dir, img_dir_lis, [df.loc[i, 'image']])

            # Predict the label for the downloaded image
            pred = get_prediction(api_url, f"{save_dir}/{img_name}")

            # Add the prediction to the lists
            ind_lis.append(i)
            pred_lis.append(pred)

            # Update the json file with the latest prediction
        update_json(json_dir, ind_lis, pred_lis)
            
    else:
        print("The next index is less than the last index in the json.")

In [ ]:
def process_df(
    next_ind,
    save_dir,
    json_dir,
    df,
    api_url
):
    """
    Fetches predictions for new images and updates the JSON file.

    Parameters:
    - next_ind (int): The next index to process.
    - api_url (str): API endpoint for image prediction.
    - save_dir (str): Directory where images are stored.
    - json_dir (str): Path to the JSON file storing predictions.
    - df (pd.DataFrame): DataFrame containing image URLs.

    Returns:
    - None: Updates the JSON file with new predictions.

    Exception Handling:
    - Ensures JSON file is valid.
    - Handles missing images and API errors.
    - Logs unexpected errors.
    """

    try:
        # Load existing image list and JSON data
        img_dir_lis = dir_image_lis(save_dir)
        json_data = load_json(json_dir)

        if not json_data:
            json_data = {'index': [], 'prediction': []}  # Initialize an empty structure
            print(f"JSON file {json_dir} was missing or empty. Creating a new one.")

        ind_lis = json_data.get("index", [])
        pred_lis = json_data.get("prediction", [])

        if not ind_lis:
            last_ind = -1  # If the JSON is empty, start from scratch
        else:
            last_ind = max(ind_lis)

        if next_ind > last_ind:
            for i in range(last_ind + 1, next_ind):
                try:
                    if i >= len(df):
                        print(f"Warning: Index {i} is out of DataFrame range.")
                        continue

                    image_link = df.loc[i, "image"]
                    img_name = os.path.basename(image_link)

                    # Check if image is already downloaded
                    if img_name not in img_dir_lis:
                        download_images(save_dir, img_dir_lis, [image_link])

                    # Predict the label for the downloaded image
                    img_path = os.path.join(save_dir, img_name)
                    pred = get_prediction(api_url, img_path)
                    # pred = "my pred"

                    # Append new data
                    ind_lis.append(i)
                    pred_lis.append(pred)

                except Exception as e:
                    print(f"Error processing index {i}: {e}")

            # Update the JSON file with the latest predictions
            update_json(json_dir, ind_lis, pred_lis)

        else:
            print("The next index is less than or equal to the last index in the JSON.")

    except Exception as e:
        print(f"Unexpected error in get_pred: {e}")

In [60]:
api_url = "http://95.217.57.146:5010/upload"

In [ ]:
process_df(
    0,
    save_dir,
    json_dir,
    df,
    api_url
)

Error: Invalid JSON format in ../data/pred.json
JSON file ../data/pred.json was missing or empty. Creating a new one.
Successfully updated ../data/pred.json


In [64]:
img = '../data/images/01fbc0d1-c4f8-4bb7-a4f8-fb3c3585dd22.jpg'
get_prediction(api_url, img)

API request error: HTTPConnectionPool(host='95.217.57.146', port=5010): Read timed out. (read timeout=50)
